In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
# Drop column with only one value
columns_list = df.columns.tolist()
columns_to_drop = [column for column in columns_list if len(df[column].unique())<2]
df_clean = df.drop(columns_to_drop, axis=1)

In [7]:
# Make a copy of df for encoding
df_encoded = df_clean.copy()

In [8]:
# Find categorical columns except issue_d and next_pymnt_d
categorical_feature_mask = df_encoded.dtypes==object
categorical_cols = df_encoded.columns[categorical_feature_mask].tolist()
categorical_cols.remove('issue_d')
categorical_cols.remove('next_pymnt_d')
categorical_cols

['home_ownership',
 'verification_status',
 'loan_status',
 'initial_list_status',
 'application_type']

In [9]:
# Custom Encoding for issue_d and next_pymnt_d
month_num = {   
   "Jan-2019": 1,
   "Feb-2019": 2,
   "Mar-2019": 3,
   "Apr-2019": 4,
   "May-2019": 5,
   "Jun-2019": 6,
   "Jul-2019": 7,
   "Aug-2019": 8,
   "Sep-2019": 9,
   "Oct-2019": 10,
   "Nov-2019": 11,
   "Dec-2019": 12,}

df_encoded["issue_d"] = df_encoded["issue_d"].apply(lambda x: month_num[x])
df_encoded["next_pymnt_d"] = df_encoded["next_pymnt_d"].apply(lambda x: month_num[x])

In [10]:
# Import labelencoder
from sklearn.preprocessing import LabelEncoder
# Instantiate labelencoder object
le = LabelEncoder()

# Apply le on categorical feature columns
df_encoded[categorical_cols] = df_encoded[categorical_cols].apply(lambda col: le.fit_transform(col))
df_encoded.head(10)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,dti,delinq_2yrs,...,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10500.0,0.1719,375.35,3,66000.0,1,3,1,27.24,0.0,...,8.0,0.0,3.0,85.7,100.0,0.0,65687.0,38199.0,2000.0,61987.0
1,25000.0,0.2000,929.09,1,105000.0,2,3,1,20.23,0.0,...,17.0,0.0,6.0,91.2,50.0,1.0,271427.0,60641.0,41200.0,49197.0
2,20000.0,0.2000,529.88,1,56000.0,2,3,1,24.26,0.0,...,8.0,0.0,4.0,66.7,50.0,0.0,60644.0,45684.0,7500.0,43144.0
3,10000.0,0.1640,353.55,3,92000.0,2,3,1,31.44,0.0,...,10.0,0.0,1.0,100.0,50.0,1.0,99506.0,68784.0,19700.0,76506.0
4,22000.0,0.1474,520.39,1,52000.0,0,3,1,18.76,0.0,...,14.0,0.0,1.0,100.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0
5,25000.0,0.1797,634.43,1,90000.0,1,3,1,18.49,1.0,...,14.0,0.0,1.0,93.1,66.7,0.0,550717.0,418934.0,18800.0,369051.0
6,20400.0,0.2000,540.48,3,51315.0,1,3,1,25.63,0.0,...,8.0,0.0,4.0,100.0,0.0,1.0,40023.0,23469.0,3500.0,27523.0
7,10000.0,0.1557,349.46,2,66000.0,1,3,1,9.00,0.0,...,11.0,0.0,3.0,100.0,16.7,0.0,29300.0,22377.0,8400.0,12500.0
8,14000.0,0.0881,443.96,3,45000.0,0,3,1,10.24,0.0,...,9.0,0.0,4.0,100.0,0.0,0.0,25100.0,9724.0,12100.0,3000.0
9,10000.0,0.1474,345.39,3,63295.0,2,3,1,39.34,0.0,...,27.0,0.0,3.0,97.2,71.4,0.0,119822.0,102924.0,15300.0,80722.0


# Split the Data into Training and Testing

In [11]:
# Create our features
X = df_encoded.drop('loan_status', axis=1)

# Create our target
y = df_encoded['loan_status']

In [12]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,dti,delinq_2yrs,inq_last_6mths,...,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,1.726172,21.778153,0.217766,0.497697,...,12.571995,0.052138,2.219423,95.057627,30.626217,0.125972,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.743862,20.199244,0.718367,0.758122,...,6.021318,0.390633,1.897432,8.326426,33.631463,0.336732,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,1.000000,0.000000,0.000000,0.000000,...,2.000000,0.000000,0.000000,20.000000,0.000000,0.000000,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,1.000000,13.890000,0.000000,0.000000,...,8.000000,0.000000,1.000000,93.000000,0.000000,0.000000,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,2.000000,19.760000,0.000000,0.000000,...,11.000000,0.000000,2.000000,100.000000,20.000000,0.000000,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,2.000000,26.660000,0.000000,1.000000,...,16.000000,0.000000,3.000000,100.000000,50.000000,0.000000,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,3.000000,999.000000,18.000000,5.000000,...,72.000000,18.000000,19.000000,100.000000,100.000000,4.000000,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [13]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [14]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [15]:
# Resample the training data with the RandomOversampler
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)
y_pred = brf.predict(X_test)

In [16]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)  

0.7717163028276635

In [17]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm_brf_df = pd.DataFrame(cm, index=['Actual 0', 'Actual 1'],
                    columns=['Predicted 0', 'Predicted 1'])
display(cm_brf_df)

,Predicted 0,Predicted 1
Actual 0,66,35
Actual 1,1882,15222


In [18]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print("Imbalanced Classification Report for BalancedRandomForestClassifier")
print(classification_report_imbalanced(y_test, y_pred))

Imbalanced Classification Report for BalancedRandomForestClassifier
                   pre       rec       spe        f1       geo       iba       sup

          0       0.03      0.65      0.89      0.06      0.76      0.57       101
          1       1.00      0.89      0.65      0.94      0.76      0.60     17104

avg / total       0.99      0.89      0.65      0.94      0.76      0.60     17205



In [19]:
# List the features sorted in descending order by feature importance
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.10102992517094041, 'total_rec_prncp'),
 (0.06690106814446464, 'total_pymnt_inv'),
 (0.06175221250761202, 'total_pymnt'),
 (0.056331075893136934, 'last_pymnt_amnt'),
 (0.04603056291364795, 'total_rec_int'),
 (0.032130937930411066, 'issue_d'),
 (0.03190370575737354, 'int_rate'),
 (0.022764672107310938, 'mths_since_recent_inq'),
 (0.020791825910062293, 'installment'),
 (0.016876248964915824, 'out_prncp'),
 (0.01635460825501234, 'out_prncp_inv'),
 (0.016052448313041954, 'total_bc_limit'),
 (0.015764747948317823, 'annual_inc'),
 (0.01546684900471776, 'bc_util'),
 (0.01458564593534848, 'avg_cur_bal'),
 (0.01435878833127368, 'mo_sin_old_rev_tl_op'),
 (0.014100377413409535, 'dti'),
 (0.013961493675348671, 'total_bal_ex_mort'),
 (0.0139133150465365, 'revol_bal'),
 (0.013875517873203642, 'bc_open_to_buy'),
 (0.01384766296733003, 'total_il_high_credit_limit'),
 (0.013813657690064261, 'max_bal_bc'),
 (0.013794875351414722, 'tot_hi_cred_lim'),
 (0.013143778105936466, 'mo_sin_old_il_acct'),
 (0.

### Easy Ensemble AdaBoost Classifier

In [20]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train, y_train)
y_pred = eec.predict(X_test)

In [21]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred) 

0.9318939348331465

In [22]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_eec_df = pd.DataFrame(cm, index=['Actual 0', 'Actual 1'],
                    columns=['Predicted 0', 'Predicted 1'])
display(cm_eec_df)

,Predicted 0,Predicted 1
Actual 0,93,8
Actual 1,975,16129


In [23]:
# Print the imbalanced classification report
print("Imbalanced Classification Report for Easy Ensemble AdaBoost Classifier")
print(classification_report_imbalanced(y_test, y_pred))

Imbalanced Classification Report for Easy Ensemble AdaBoost Classifier
                   pre       rec       spe        f1       geo       iba       sup

          0       0.09      0.92      0.94      0.16      0.93      0.87       101
          1       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205

